In [1]:
import pandas as pd
import numpy as np

import xgboost as xgb
from hyperopt import fmin, tpe, hp, STATUS_OK, Trials
from functools import partial
import requests
import os
import time
import geopandas as gpd

import datetime
import pytz

import seaborn as sns
import matplotlib.pyplot as plt

from math import sin, cos, sqrt, atan2, radians
from tqdm import tqdm

from catboost import CatBoostRegressor

from sklearn.neighbors import KNeighborsClassifier, KNeighborsRegressor
from sklearn.metrics import mean_squared_error, roc_auc_score

from sklearn.linear_model import Ridge

from sklearn.model_selection import train_test_split, KFold, StratifiedKFold
from sklearn.feature_selection import SelectFromModel

from zipfile import ZipFile
from tzwhere import tzwhere

%matplotlib inline

In [2]:
def rmse(y_true, y_pred):
    return sqrt(mean_squared_error(y_true, y_pred))

In [3]:
def load_geo_dataframe(filename, path='data'):
    """ load geo dataframe with point geometries from files """
    
    df = None
    for f in tqdm(os.listdir(path)):
        if os.path.isdir(os.path.join(path,f)):
            if df is None:
                df = gpd.read_file(os.path.join(path, f, filename))
            else:
                df = df.append(gpd.read_file(os.path.join(path, f, filename)))

    df['lat'] = df.geometry.y
    df['long'] = df.geometry.x
    
    return df

In [4]:
R = 6373.0 # радиус земли в километрах

def distance(x,y):
    """
    Параметры
    ----------
    x : tuple, широта и долгота первой геокоординаты 
    y : tuple, широта и долгота второй геокоординаты 
    
    Результат
    ----------
    result : дистанция в километрах между двумя геокоординатами
    """
    lat_a, long_a, lat_b, long_b = map(radians, [*x,*y])    
    dlon = long_b - long_a
    dlat = lat_b - lat_a
    a = sin(dlat/2)**2 + cos(lat_a) * cos(lat_b) * sin(dlon/2)**2
    c = 2 * atan2(sqrt(a), sqrt(1 - a))
    return R * c

def nearest_distances(data, initial_data=None, metric=distance, n_neighbors=10, return_more=None):
    
    start_time = time.time()
    knc = KNeighborsClassifier(metric=metric)
    dots = data[['lat','long']].dropna()
    knc.fit(X=dots , y=np.ones(dots.shape[0]))
    if initial_data is None:
        distances, indexes = knc.kneighbors(X=dots, n_neighbors=n_neighbors)
    else:
        distances, indexes = knc.kneighbors(X=initial_data, n_neighbors=n_neighbors)
        
    print('Finded neares distances in {}'.format(time.time() - start_time))
    if return_more:
        more = data[[return_more]].values[indexes]
        return distances, more
    else:
        return distances

def timezone_from_coordinates(lat, long, tzw):

    return tzw.tzNameAt(lat, long)

In [5]:
pois = load_geo_dataframe('gis_osm_pois_free_1.shp')
transport = load_geo_dataframe('gis_osm_transport_free_1.shp')
traffic = load_geo_dataframe('gis_osm_traffic_free_1.shp')
cities = load_geo_dataframe('gis_osm_places_free_1.shp')

100%|██████████████████████████████████████████████████████████████████████████████████| 16/16 [00:11<00:00,  1.45it/s]


In [6]:
osm_categories = ['pharmacy','supermarket','clothes','doityourself','hairdresser','post_office', 'car_wash',
                  'beverages','florist','mobile_phone_shop','beauty_shop','dentist','school','hospital','doctors',
                  'furniture_shop','butcher','bakery','artwork','kiosk','shoe_shop','car_dealership','jeweller',
                 'chemist','greengrocer','toy_shop','gift_shop','bookshop','optician','sports_shop','vending_any',
                 'travel_agent','newsagent','stationery','veterinary','department_store','hostel','vending_parking',
                 'outdoor_shop','laundry','motel','bicycle_shop','university','college','hunting_stand','hotel',
                 'restaurant','fast_food','bar','nightclub','food_court','bycicle_rental','cinema','theatre','arts_centre',
                  'swimming_pool','museum','sports_centre','theme_park','zoo','stadium','mall']


traffic_categories = ['fuel', 'parking', 'parking_underground', 'service']

In [7]:
banks = pois[pois.fclass=='bank']
atms = pois[pois.fclass=='atm']
points = pois[pois.fclass.isin(osm_categories)]
stops = transport[transport.fclass.isin(['bus_station','ferry_terminal'])]
transport = transport[transport.fclass.isin(['railway_halt', 'railway_station'])]
traffic = traffic[traffic.fclass.isin(traffic_categories)]

In [8]:
train = pd.read_csv('train.csv', index_col=0)
test = pd.read_csv('test.csv', index_col=0)

In [9]:
coords = pd.read_csv('fixed_yandex_coordinates.csv')

In [10]:
# please, use YOUR Yandex API key
# usage examples for geocoder https://tech.yandex.ru/maps/doc/geocoder/desc/examples/geocoder_examples-docpage/
API_KEY = '6aadbc7c-f316-4d6d-8601-1d0fb1f69e22'

In [11]:
train.address = [' '.join(adr.split()) for adr in train.address]
test.address = [' '.join(adr.split()) for adr in test.address]

coords.address = [' '.join(adr.split()) for adr in coords.address]

In [12]:
hot_fix = {'AVIAITSIONNAYA, 12 Komsomolsk-31': [50.263251, 137.476056, ',,Комсомольск-31'], 
          'N.TERNOVSKAYA, 1 Penza': [53.132400, 45.022825, ',,Пенза'], 
          'MSK KIEVSKOE 1 A MOSKVA': [55.625801, 37.439239, ',,Москва'], 
          'D. 2, UL. KRASNAYA GURYEVSK G': [54.770972, 20.606706, ',,Гурьевск'], 
          'MOZHAYSKOE SH.,80A Odintsovo': [55.660353, 37.188923, ',,Одинцово'], 
          'KOMSOM NA AMURE 37 Komsomolsk-31': [50.263251, 137.476056, ',,Комсомольск-31'], 
          'UL. KUTUZOVA VOLOGDA 20 N': [59.157670, 38.721192, ',,Вологда-20']}

In [13]:
for fix in hot_fix.keys():
    coords.loc[coords.address==fix, 'lat'] = hot_fix[fix][0]
    coords.loc[coords.address==fix, 'long'] = hot_fix[fix][1]
    coords.loc[coords.address==fix, 'address_rus'] = hot_fix[fix][2]

In [14]:
train_data = train.loc[train.lat.notnull(), :].append(train.loc[train.lat.isnull(), :][[
            'id', 'atm_group', 'address','target']].merge(coords, on='address', how='left'), sort=False).reset_index(drop=True)

test_data = test.loc[test.lat.notnull(), :].append(test.loc[test.lat.isnull(), :][[
            'id', 'atm_group', 'address']].merge(coords, on='address', how='left'), sort=False).reset_index(drop=True)

In [15]:
target = train_data['target']
train_data_copy = train_data.copy()
train_data = train_data.drop('target', axis=1)

In [16]:
X = train_data.append(test_data).reset_index(drop=True)

In [17]:
distances = nearest_distances(data=X, n_neighbors=6)

distances_osm = nearest_distances(data=points, initial_data=X[['lat','long']].dropna(), n_neighbors=10)

distances_cities = nearest_distances(data=cities, initial_data=X[['lat','long']].dropna(), n_neighbors=10)
distances_transport = nearest_distances(data=transport, initial_data=X[['lat','long']].dropna(), n_neighbors=10)
distances_traffic = nearest_distances(data=traffic, initial_data=X[['lat','long']].dropna(), n_neighbors=10)
distances_bank = nearest_distances(data=banks, initial_data=X[['lat','long']].dropna(), n_neighbors=10)
distances_atm = nearest_distances(data=atms, initial_data=X[['lat','long']].dropna(), n_neighbors=10)
distances_stops = nearest_distances(data=stops, initial_data=X[['lat','long']].dropna(), n_neighbors=10)

Finded neares distances in 33.604456186294556
Finded neares distances in 224.02339029312134
Finded neares distances in 217.73690629005432
Finded neares distances in 52.30057907104492
Finded neares distances in 142.40742945671082
Finded neares distances in 111.70325136184692
Finded neares distances in 37.75513434410095
Finded neares distances in 44.126532793045044


In [18]:
distances_group = []
for gr in X.atm_group.unique():
    distances_group.append(nearest_distances(data=X[X.atm_group==gr], initial_data=X[['lat','long']].dropna(), n_neighbors=6))

Finded neares distances in 2.815467596054077
Finded neares distances in 18.242207527160645
Finded neares distances in 6.844727516174316
Finded neares distances in 17.898127555847168
Finded neares distances in 11.44852089881897
Finded neares distances in 28.485808849334717
Finded neares distances in 14.868214845657349


In [19]:
def add_distance_features(df, distances, feature_prefix):
    df_copy = df.copy()
    
    for n in range(distances.shape[1]):
        df_copy['{}_{}'.format(feature_prefix, n)] = distances[:, n]
        
    return df_copy

def add_stat_distance_features(df, feature_prefix):
    df_copy = df.copy()
    
    df_copy['{}_mean'.format(feature_prefix)] = df_copy.iloc[:, df_copy.columns.str.contains(feature_prefix)].mean(axis=1)
    # df_copy['{}_median'.format(feature_prefix)] = df_copy.iloc[:, df_copy.columns.str.contains(feature_prefix)].median(axis=1)
    df_copy['{}_std'.format(feature_prefix)] = df_copy.iloc[:, df_copy.columns.str.contains(feature_prefix)].std(axis=1)
    df_copy['{}_interquartile'.format(feature_prefix)] = df_copy.iloc[:, df_copy.columns.str.contains(feature_prefix)].quantile(0.75, axis=1) - \
                               df_copy.iloc[:, df_copy.columns.str.contains(feature_prefix)].quantile(0.25, axis=1)
        
    df_copy['{}_min_max'.format(feature_prefix)] = df_copy.iloc[:, df_copy.columns.str.contains(feature_prefix)].max(axis=1) - \
                         df_copy.iloc[:, df_copy.columns.str.contains(feature_prefix)].min(axis=1)
        
    return df_copy

In [20]:
X_feat = add_distance_features(X[['lat','long']].dropna(), distances, 'distance_data')

X_feat = add_distance_features(X_feat, distances_osm, 'distance_osm')

X_feat = add_distance_features(X_feat, distances_transport, 'distance_transport')
X_feat = add_distance_features(X_feat, distances_cities, 'distance_cities')
X_feat = add_distance_features(X_feat, distances_traffic, 'distance_traffic')
X_feat = add_distance_features(X_feat, distances_bank, 'distance_bank')
X_feat = add_distance_features(X_feat, distances_atm, 'distance_atm')
X_feat = add_distance_features(X_feat, distances_stops, 'distance_stops')

for pref in ['distance_data','distance_osm','distance_transport',
             'distance_cities','distance_traffic','distance_bank','distance_atm','distance_stops']:
    X_feat = add_stat_distance_features(X_feat, pref)

In [21]:
for n, gr in enumerate(X.atm_group.unique()):
    X_feat = add_distance_features(X_feat, distances_group[n], 'distance_group_{}'.format(gr))
    X_feat = add_stat_distance_features(X_feat, 'distance_group_{}'.format(gr))

In [22]:
X = X.merge(X_feat.drop_duplicates(subset=['lat','long']), on=['lat','long'], sort=False, how='left')

In [23]:
X['city'] = X[~X.address_rus.isnull()].address_rus.apply(lambda x: x.split(',')[2]) 
X['eng_city'] = [i.split(' ')[-1].lower() for i in X.address]

In [24]:
rare_cities = X.city.value_counts()[(X.city.value_counts() < 10) == True].index
rare_eng_cities = X.eng_city.value_counts()[(X.eng_city.value_counts() < 10) == True].index

In [25]:
X.city = X.city.apply(lambda x: 'RARE' if x in rare_cities else x)
X.eng_city = X.eng_city.apply(lambda x: 'RARE' if x in rare_eng_cities else x)

In [26]:
X.city = X.city.rank().fillna(-1)
X.eng_city = X.eng_city.rank().fillna(-1)

In [27]:
X_ = X[:len(train_data)]

X_ = X_.drop(['address', 'address_rus','distance_data_0'], axis=1)

Y_ = target

In [33]:
kf = KFold(n_splits=10, random_state=1, shuffle=True)
results = []
models = []

test_ys = []
predictions = []

n = 0


for train_index, test_index in kf.split(X_):
    
    print("{} Fold".format(n))
    
    X_train, X_valid = X_.loc[train_index, :], X_.loc[test_index, :]
    Y_train, Y_valid = Y_[train_index], Y_[test_index]
    
    test_ys.append(np.stack([X_valid['id'].values, Y_valid.values], axis=0))

    gbm = xgb.XGBRegressor(**{'alpha': 10.0, 'colsample_bytree': 1.0, 'eta': 0.2, 'gamma': 0.0, 'lambda': 0.9, 
                              'learning_rate': 0.03, 'max_depth': 11, 'min_child_weight': 10.0, 'n_estimators': 400, 
                              'subsample': 0.8, 'random_state': 1, 'n_jobs': -1})
    
    gbm1 = xgb.XGBRegressor(**{'alpha': 10.0, 'colsample_bytree': 1.0, 'eta': 0.2, 'gamma': 0.0, 'lambda': 0.9, 
                              'learning_rate': 0.03, 'max_depth': 11, 'min_child_weight': 10.0, 'n_estimators': 400, 
                              'subsample': 0.8, 'random_state': 4, 'n_jobs': -1})
    
    gbm2 = xgb.XGBRegressor(**{'alpha': 10.0, 'colsample_bytree': 1.0, 'eta': 0.2, 'gamma': 0.0, 'lambda': 0.9, 
                              'learning_rate': 0.03, 'max_depth': 11, 'min_child_weight': 10.0, 'n_estimators': 400, 
                              'subsample': 0.8, 'random_state': 9, 'n_jobs': -1})

    gbm.fit(X_train, Y_train, eval_set=[(X_valid, Y_valid)], early_stopping_rounds=20, eval_metric='rmse',
            verbose=False)
    gbm1.fit(X_train, Y_train, eval_set=[(X_valid, Y_valid)], early_stopping_rounds=20, eval_metric='rmse',
            verbose=False)
    gbm2.fit(X_train, Y_train, eval_set=[(X_valid, Y_valid)], early_stopping_rounds=20, eval_metric='rmse',
            verbose=False)
    
    
    models.append(gbm)
    predictions.append(gbm.predict(X_valid))
    models.append(gbm1)
    predictions.append(gbm1.predict(X_valid))
    models.append(gbm2)
    predictions.append(gbm2.predict(X_valid))

    results.append(rmse(Y_valid, np.mean([gbm.predict(X_valid), gbm1.predict(X_valid), gbm2.predict(X_valid)], axis=0)))

    n += 1
    
print("results: {}, mean_result: {:.5f}, std: {:.4f}".format(results, np.mean(results), np.std(results)))

0 Fold
1 Fold
2 Fold
3 Fold
4 Fold
5 Fold
6 Fold
7 Fold
8 Fold
9 Fold
results: [0.04193752242045955, 0.042053930097359, 0.041223266908563434, 0.04144114205972502, 0.041061398889181616, 0.04286462683382677, 0.04348723520218232, 0.0419338362252758, 0.04214691844699118, 0.04234620929009307], mean_result: 0.04205, std: 0.0007


In [ ]:
# fig, ax = plt.subplots(figsize=(10,40))
# xgb.plot_importance(gbm, ax=ax)

In [34]:
X_test = X[len(train_data):].drop(['address', 'address_rus', 'distance_data_0'], axis=1).sort_values(by='id')

In [35]:
submit = pd.DataFrame(np.mean([models[n].predict(X_test) for n in range(len(models))], axis=0), 
                      index=test.sort_values(by='id').index, columns=['target'])

In [36]:
submit.sort_index().to_csv('submit_not_fixed.csv')

In [37]:
# we have a little leak - mean of train target and predicted target is 0
# and std of train targets and predicted targets must be 0.1 (because we divide by 10 our target)
submit.target.mean(), np.mean(target), np.std(list(submit.target * 10) + list(target * 10))

(-0.00032762368, 0.0007147964950465309, 0.8223676156547468)

In [38]:
# let substract bias in train data
submit.target = submit.target - (np.mean(target) + submit.target.mean())

In [39]:
submit.sort_index().to_csv('submit_fixed.csv')